# Interactive masking

In this example, we will use a custom drawing tool to draw rectangles on a 2D figure.
The data inside the rectangles will be masked.

In [ ]:
%matplotlib widget
import plopp as pp
import scipp as sc

We first generate some data that contains three bands of peaks that all have different spreads.

In [ ]:
from plopp.data.examples import three_bands

da = three_bands()

We then construct our custom tool,
using Mpltoolbox's [Rectangles tool](https://mpltoolbox.readthedocs.io/en/latest/rectangles.html),
and inheriting from Plopp's
[DrawingTool](https://scipp.github.io/plopp/about/generated/plopp.widgets.drawing.DrawingTool.html).

In [ ]:
from plopp.widgets.drawing import DrawingTool
from functools import partial
from mpltoolbox import Rectangles


def define_mask(da, rect_info):
    """
    Function that creates a mask inside the area
    covered by the rectangle.
    """
    x = rect_info['x']
    y = rect_info['y']
    b = min(y['bottom'], y['top'])
    t = max(y['bottom'], y['top'])
    l = min(x['left'], x['right'])
    r = max(x['left'], x['right'])

    xcoord = sc.midpoints(da.coords[x['dim']])
    ycoord = sc.midpoints(da.coords[y['dim']])
    return (xcoord >= l) & (xcoord <= r) & (ycoord >= b) & (ycoord <= t)


def _get_rect_info(artist, figure):
    """
    Convert the raw rectangle info to a dict containing the dimensions of
    each axis, and values with units.
    """
    return lambda: {
        'x': {
            'dim': figure.canvas.dims['x'],
            'left': sc.scalar(artist.xy[0], unit=figure.canvas.units['x']),
            'right': sc.scalar(
                artist.xy[0] + artist.width, unit=figure.canvas.units['x']
            ),
        },
        'y': {
            'dim': figure.canvas.dims['y'],
            'bottom': sc.scalar(artist.xy[1], unit=figure.canvas.units['y']),
            'top': sc.scalar(
                artist.xy[1] + artist.height, unit=figure.canvas.units['y']
            ),
        },
    }


RectangleTool = partial(
    DrawingTool, tool=Rectangles, get_artist_info=_get_rect_info, icon='vector-square'
)

Finally, we create our visualization interface with the figure,
adding our new tool to the toolbar.

In [ ]:
data_node = pp.Node(da)


def apply_masks(da, *masks):
    out = da.copy(deep=False)
    for i, mask in enumerate(masks):
        out.masks[str(i)] = mask
    return out


masking_node = pp.Node(apply_masks, data_node)

fig = pp.imagefigure(masking_node, norm='log')

r = RectangleTool(
    figure=fig, input_node=data_node, func=define_mask, destination=masking_node
)
fig.toolbar['roi'] = r

In [ ]:
r.value = True
r._tool.click(50, 200)
r._tool.click(200, 250)
r._tool.click(30, 50)
r._tool.click(250, 170)

In [ ]:
fig

In [ ]:
pp.show_graph(fig)

To retrieve the masked data array,
simply call the node that is applying the masks:

In [ ]:
masking_node()

In [ ]:
# This cell is used to generate the thumbnail for the docs gallery.
# It is hidden from the online documentation.
fig.save('../_static/gallery/interactive-masking-thumbnail.png')